# Backtracking

Many computational problems can be solved by systematically searching over the set of possible solutions:

* to solve a maze, you can try every possible path from the starting point
* to solve a sudoku puzzle, you can set the first empty square to a 1, and then try to solve the rest of the
puzzle
    * if you don't find a solution, then undo everything and set the first empty square to a 2, and then try
    to solve the rest of the puzzle
        * repeat the process until you find a solution
* 

Backtracking is a possible strategy when the solution to a problem involves one or more steps, and at each
step there is one or more choices that can be made. Backtracking builds a solution recursively by attempting
all of the choices at each step. If a point is reached where no solution is possible, then we backtrack to
a previous step and make a different choice.

## The JumpIt problem

We begin by examining a problem that has a simple recursive solution that does not use backtracking, and then
we examine a variation of the problem that requires backtracking.

The JumpIt problem is a game played on a linear board made up of $n \geq 2$ squares where each square is
marked with a non-negative number equal to the cost of stepping on the square. The game starts on the
leftmost square. On each move, you can move one or two squares to the right with the goal of reaching the
rightmost square. The problem is to find the sequence of moves having the lowest total cost.

The following figure shows an example of a small JumpIt board.

![](week11/jumpit1.png)

The reader can easily verify that the sequence of moves producing the smallest cost is to move 1 square, then 
2 squares, and then 2 squares for a total cost of $0 + 3 + 6 + 10 = 19$:

![](week11/jumpit2.png)

The solution in the above figure corresponds to choosing to move to the next square having the lowest cost. Such
strategies are called *greedy strategies* because they make a decision by choosing the best next move without
considering whether such a decision in fact leads to best overall solution. In the JumpIt problem, a greedy
strategy does not always lead to the lowest cost as illustrated in the following example:

![](week11/jumpit3.png)

**Exercise 1** Why does the greedy strategy sometimes not produce the sequence of moves having the lowest
total cost?

<div class="alert alert-info">
    The greedy strategy sometimes forces you to move to square that can be skipped.
</div>

### A recursive solution

We would like to write a recursive function `cost(board)` that returns the lowest cost for the given `board`. A
recursive solution is possible because we know that the total cost is equal to the cost of the leftmost square
plus the *minimum* of two possible values:

* `cost(<board after moving 1 square>)`
* `cost(<board after moving 2 squares>)`

There are two base cases to consider. The first base case occurs for the smallest possible board size of
$n=2$ where the total cost is simply the sum of the costs of the two squares. The second base occurs for a
board size of $n=3$ where the total cost is the sum of the costs of the first and last squares.

A recursive solution for finding the smallest total cost for JumpIt is:

In [ ]:
def cost(board):
    n = len(board)
    if n == 2:
        return board[0] + board[1]
    elif n == 3:
        return board[0] + board[2]
    else:
        c1 = board[0] + cost(board[1:])
        c2 = board[0] + cost(board[2:])
        return min(c1, c2)
    
board = [0, 3, 80, 6, 57, 10]
print(cost(board))
board = [17, 1, 5, 6, 1]
print(cost(board))

**Exericse 2** Rewrite `cost` so that it avoids slicing or copying the board. You will need to introduce a
helper method.

In [ ]:
def cost(board):
    n = len(board)
    if n == 2:
        return board[0] + board[1]
    elif n == 3:
        return board[0] + board[2]
    else:
        c1 = board[0] + cost(board[1:])
        c2 = board[0] + cost(board[2:])
        return min(c1, c2)

def cost_impl(board, index):
    # index the current position on the board
    n = len(board)
    if index == n - 2:
        return board[n - 2] + board[n - 1]
    elif index == n - 3:
        return board[n - 3] + board[n - 1]
    else:
        c1 = board[index] + cost_impl(board, index + 1)
        c2 = board[index] + cost_impl(board, index + 2)
        return min(c1, c2)
    
board = [0, 3, 80, 6, 57, 10]
print(cost(board))
board = [17, 1, 5, 6, 1]
print(cost(board))

**Exercise 3** Why is the base case for $n=3$ required? What happens if you remove it? Can you replace it with
a different base case?

<div class="alert alert-info">
    It represents the case where you can move two squares to reach the end of the board. If you simply remove it,
    then an IndexError is raised. You can replace it with a base case where the board size is 1 (return board[0])
</div>

## A variation of JumpIt

Consider the following variation of JumpIt played on a board such as the one shown below:

![](week11/board1.png)

Starting from the leftmost square, we want to know if it is possible to reach the rightmost square (which always
has the number 0 on it). You can move
$m$ squares left or right from the current square where $m$ is equal to the number on the square, but you cannot
move off of the board; for example, on the board shown above, the first move must be two squares to the right.

The board shown above has at least two different solutions:

| Starting square | Move direction | Landing square |
| :- | :- | :- |
| 2 | right | 4 |
| 4 | right | 1 |
| 1 | right | 6 |
| 6 | left | 3 |
| 3 | right | 5 |
| 5 | right | 0 |

| Starting square | Move direction | Landing square |
| :- | :- | :- |
| 2 | right | 4 |
| 4 | right | 1 |
| 1 | left | 2 |
| 2 | left | 2 |
| 2 | left | 3 |
| 3 | right | 5 |
| 5 | right | 0 |

Unlike the original version of JumpIt, it is possible to find cycles where a move takes you back to square
that you have already visited; for example, a cycle exists between the two squares marked with a 2:

![](week11/board2.png)

It is possible to create boards with longer cycles. For example, if we change the square marked with a 3 to a 1
then it is possible to create a sequence of moves that returns to the starting leftmost square.

On the following board, every sequence of moves eventually leads to a cycle which means that no solution
exists:

![](week11/board3.png)

Even without cycles, it is easy to create a board where no solution exists:

![](week11/board4.png)

From the above discussion, we can observe that when considering a sequence of moves we can reject the sequence
as a candidate solution if it:

* visits a square that has already been visited, or
* visits a square where there are no legal moves

To determine if a square has been previously visited, we require someway to record which squares have been
visited. We could create a list of visited squares where we store the index of the visited squares, but this
requires additional memory of order $O(n)$ if there are $n$ squares in the board. An alternative approach is
to modify the board to indicate that a square has been visited. We can do this by changing the value of the
number on a square by negating the value; all visited squares will then have a negative value.

## A general backtracking algorithm

A backtracking alorithm explores all possible candidate solutions but abandons (or backtracks) a candidate solution
when it can determine that the candidate does not lead to a solution. A general backtracking algorithm has the
following structure:

```
explore(scenario):
    if a solution is reached:
        return the solution
    else:
        for each choice:
            if the choice is not a dead end:
                make the choice
                solution = explore(updated scenario)
                if there is a solution:
                    return the solution
                else:
                    undo the choice
        return no-solution
```

In the algorithm shown above, the `scenario` is all of the information needed to describe the current state of
a candidate solution. In our problem, the `scenario` would consist of a board and the current location
(index) on the board.

We know that a solution has been obtained if the current location on the board is the last square of the board.

For any square of the board, there are two possible choices for the next move: We can move $m$ squares to the
left or we can move $m$ squares to the right where $m$ is the value of the current square.

A choice is a dead end if it results in moving off of the board or if it moves to an already visited square.

Making a choice means moving $m$ squares left or right. To make the choice, we:

1. update the square that we are currently on by negating its value (to indicate that we have visited the square)
2. create a variable `next_loc` equal to the index of the square that we have chosen to move to

`next_loc` is passed to the recursive call as part of the `updated scenario`.

To undo a choice, we:

1. unmark the square that we are currently on by negating its value (making it positive to indicate that the
square has not been visited)

The following cell contains an implementation of the `explore` algorithm shown above. `explore` is called
as a recursive helper function by the function `is_solvable`. A separate function `is_dead_end` implements
the tests that determine if a scenario is a dead end (does not lead to a solution).

In [ ]:
def is_dead_end(board, loc):
    # is loc off of the board?
    if loc < 0 or loc >= len(board):
        return True
    # is loc already visited?
    if board[loc] < 0:
        return True
    return False
    

def explore(board, curr_loc):
    n = len(board)
    if curr_loc == n - 1:
        return True
    else:
        # value of square we are currently on
        m = board[curr_loc]
        # choices are the indexes of the squares m spots to the left and right
        choices = [curr_loc - m, curr_loc + m]
        for choice in choices:
            if not is_dead_end(board, choice):
                # make the choice
                board[curr_loc] = -board[curr_loc]
                next_loc = choice
                # explore the choice
                has_solution = explore(board, next_loc)
                if has_solution:
                    return True
                else:
                    # undo the choice
                    board[curr_loc] = -board[curr_loc]
        return False


def is_solvable(board):
    return explore(board, 0)


board = [2, 3, 4, 2, 5, 2, 1, 6, 9, 0]
print(is_solvable(board))
board = [2, 1, 2, 2, 2, 2, 2, 6, 3, 0]
print(is_solvable(board))
board = [1, 100, 2, 0]
print(is_solvable(board))

**Exercise 4** Modify the function `explore` so that it returns a list of the indexes of the squares that are
moved to during a solution, or an empty list if there is no solution.

In [ ]:
def is_dead_end(board, loc):
    # is loc off of the board?
    if loc < 0 or loc >= len(board):
        return True
    # is loc already visited?
    if board[loc] < 0:
        return True
    return False
    

def explore(board, curr_loc):
    n = len(board)
    if curr_loc == n - 1:
        return [curr_loc]    # solved, no more moves
    else:
        indexes = []
        # value of square we are currently on
        m = board[curr_loc]
        # choices are the indexes of the squares m spots to the left and right
        choices = [curr_loc - m, curr_loc + m]
        for choice in choices:
            if not is_dead_end(board, choice):
                # make the choice
                indexes.append(choice)                     # adds choice to list of moves
                board[curr_loc] = -board[curr_loc]
                next_loc = choice
                # explore the choice
                rest_of_indexes = explore(board, next_loc)
                if len(rest_of_indexes) != 0:
                    indexes.extend(rest_of_indexes)
                    return indexes
                else:
                    # undo the choice
                    indexes.pop()                          # removes choice from list of moves
                    board[curr_loc] = -board[curr_loc]
        return indexes


def is_solvable(board):
    indexes = explore(board, 0)
    if indexes:
        # Explore has the quirk that it includes the rightmost square twice.
        # There is a way to fix this but it involves changing the structure
        # of the general backtracking algorithm, so instead I just remove the
        # duplicated square.
        indexes.pop()
    return indexes


board = [2, 3, 4, 2, 5, 2, 1, 6, 9, 0]
print(is_solvable(board))
board = [2, 1, 2, 2, 2, 2, 2, 6, 3, 0]
print(is_solvable(board))
board = [1, 100, 2, 0]
print(is_solvable(board))

## The eight queens puzzle

The [eight queens puzzle](https://en.wikipedia.org/wiki/Eight_queens_puzzle) is a very common problem used
to teach recursive backtracking. The goal of the puzzle is to place eight queens on a chessboard so that no
queen is attacking another queen:

![](week11/queens.png)

In chess, a queen can move any number of squares horizontally, vertically, or diagonally. A solution for the
eight queens puzzle places the queens so that no row, column, or diagonal has more than one queen.

One way to solve the puzzle is to start by placing one queen and then building the solution one queen at a time
until all eight queens are successfully placed. Because two queens cannot appear in the same row or column, we
can start by placing the first queen in column 0, followed by the second queen in column 1, followed by 
the third queen in column 2, and so on.

To use the general backtracking algorithm, we need to determine:

* how to define the scenario
* when is a solution reached
* what choices exist when placing a queen
* if a choice is a dead end
* how to make a choice
* how to undo a choice

The scenario is described the locations of the currently placed queens. If we imagine the board to be an
$8 \times 8$ grid where each row and column is numbered from 0 to 7, then we can use a list `queens` containing the
row indexes of each queen

A solution is reached when all eight queens are successfully placed.

When placing a queen in column $i$, we have a choice of eight rows to place the queen in. Therefore, our choices
are the eight indexes 0, 1, 2, ..., 8.

A choice of row is a dead end if there is another queen in the same row or on a diagonal location relative to 
the choice. Finding the diagonal locations relative to another location is the only tricky part of this problem.

To make a choice, we add the chosen row index to the end of the list `queens`.

To undo a choice, we remove the last element from the list `queens`.

The cell below contains an implementation of the general backtracking algorithm that solves the eight queens
puzzle. `explore` does not return a list of queen locations; instead, it fills in the list `queens` with the
row index of each queen so that `queens[i]` contains the row index of the queen in column `i`.

In [ ]:
def is_dead_end(queens, row):
    # is there another queen in the specified row?
    if row in queens:
        return True
    n = len(queens)
    # offset is the change in row index for a column somewhere
    # to the left of the next queen to be placed
    offset = 1
    for col in range(n - 1, -1, -1):
        # the row index of the queen in col
        r = queens[col]
        # is r diagonal to row?
        if r == row + offset or r == row - offset:
            return True
        offset = offset + 1
    return False
    
def explore(queens):
    n = len(queens)
    if n == 8:
        return
    else:
        for choice in range(0, 8):
            if not is_dead_end(queens, choice):
                # make the choice by placing the next queen in row choice
                queens.append(choice)
                # explore the choice
                explore(queens)
                if len(queens) == 8:
                    # placed all eight queens, done
                    return
                else:
                    # undo choice by removing the previously placed row
                    queens.pop()

def eight_queens():
    queen_locs = []
    explore(queen_locs)
    return queen_locs

print(eight_queens())

**Exercise 5** `explore` always produces the list `[0, 4, 7, 5, 2, 6, 1, 3]` because the row choices are always
in the order 0, 1, 2, ..., 7. Modify `explore` so that the row choices are shuffled in a random order.

<div class="alert alert-info">
    Before the loop, make a list called choices containing the numbers 0 through 7, and then shuffle the list.
    Replace the 'for choice in range(0, 8)' loop with 'for choice in choices'.
</div>

**Exercise 6** The [subset sum problem](https://en.wikipedia.org/wiki/Subset_sum_problem) is a well known
problem in computing science that can be solved using backtracking. Given a list of positive integer values and
a positive integer target value $t$, is there a subset of the values in the list that sum exactly to $t$?
Formulate a recursive backtracking solution for the subset sum problem. You may assume that the list of
values is sorted from largest to smallest value and that the values in the list are unique.

<div class="alert alert-info">
    If we use a slicing approach, then the scenario is described by the current sum we are considering, the list
    of remaining numbers that might be included in the sum, and the target sum value t.
    <br /><br />
    A solution is reached when the current sum is equal to the target value t, or if there are no more numbers
    remaining to add to the current sum.
    <br /><br />
    The choice that we make at each recursive step is to include or exclude the first element of the list in the
    sum.
    <br /><br />
    A choice leads to a dead end if the current sum plus the choice is greater than the target value t (this
    only works if the list is sorted in ascending order).
    <br /><br />
    To include the choice in the sum, we simply add the choice to the current sum.
    <br /><br />
    To undo the choice, we simply subtract the choice from the current sum.
</div>

In [ ]:
def is_dead_end(current_sum, next_value, target_sum):
    return current_sum + next_value > target_sum
    
def explore(current_sum, values, target_sum):
    if current_sum == target_sum:
        return True
    elif len(values) == 0:
        return False
    else:
        # our choices are to include values[0] in the sum, or not include it in the sum
        # we can represent the 'not include it in the sum' with a value of 0
        choices = [values[0], 0]
        for choice in choices:            
            if not is_dead_end(current_sum, choice, target_sum):
                # make the choice by adding choice to current_sum
                current_sum += choice
                # explore the choice
                # we slice the list because we are done considering the first element of the list
                has_sum = explore(current_sum, values[1:], target_sum)
                if has_sum:
                    # found a solution
                    return True
                else:
                    # undo choice by subtrating choice from current_sum
                    current_sum -= choice
        return False

def has_subset_sum(values, target_sum):
    current_sum = 0
    return explore(current_sum, values, target_sum)

values = [1, 2, 5, 6]
# True cases
for target_sum in range(1, 16):
    print(target_sum, ':', has_subset_sum(values, target_sum))

**Exercise 7** The mini-Sudoku problem requires you to arrange the numbers 1, 1, 1, 2, 2, 2, 3, 3, 3 on a
$3 \times 3$ grid so that no number is repeated in any row or column. An example of a solvd mini-Sudoku
puzzle is:

```
1 2 3
3 1 2
2 3 1
```

Given an incomplete $3 \times 3$ array of values `arr`, write a recursive backtracking function that fills in the
missing values to complete the mini-Sudoku puzzle. Use the value 0 to indicate a missing value; for example,
if `arr` is defined as:

```python
arr = [[0, 0, 0],
       [0, 0, 2],
       [0, 3, 1]]
```

then your solution should replace the zeros with the appropriate values to complete the puzzle.

<div class="alert alert-info">
    The following solution assumes that the puzzle is in fact solvable.
    <br /><br />
    The scenario is described by a two-dimensional list containing the values of the puzzle and the current
    row index and column index of the puzzle.
    <br /><br />
    A solution is reached when all of the values of the puzzle are filled in (non-zero).
    <br /><br />
    The choice that we make at each recursive step is to replace a zero valued element with the number 
    1, 2, or 3.
    <br /><br />
    A choice leads to a dead end if a value equal to the choice is already in the same row or column of the puzzle.
    <br /><br />
    To make the choice, we replace the zero valued element with the choice.
    <br /><br />
    To undo the choice, we replace the choice with a zero. We have to undo the choice if the next square
    cannot be filled in after making the choice.
</div>

In [ ]:
def is_dead_end(puzzle, row_index, col_index, choice):
    # the row of the puzzle at row_index
    row = puzzle[row_index]
    # column of the puzzle at col_index
    col = [puzzle[0][col_index], puzzle[1][col_index], puzzle[2][col_index]]
    #print('is_dead_end:', row, col, choice, row.count(choice), col.count(choice))
    if choice in row or choice in col:
        return True
    return False
    
def explore(puzzle, row_index, col_index):
    if row_index == 3:
        return True
    elif puzzle[row_index][col_index] != 0:
        # this value is already filled in, try filling the puzzle at the
        # next square
        if col_index < 2:
            # go to next column in this row
            return explore(puzzle, row_index, col_index + 1)
        else:
            # go to start of next row
            return explore(puzzle, row_index + 1, 0)
    else:
        # our choices are the values 1, 2, and 3
        choices = [1, 2, 3]
        for choice in choices:            
            if not is_dead_end(puzzle, row_index, col_index, choice):
                # make the choice by replacing puzzle[row_index][col_index] with the choice
                puzzle[row_index][col_index] = choice
                # explore the choice by trying to fill in the rest of the puzzle
                next_row = row_index
                next_col = col_index
                if col_index < 2:
                    # go to next column in this row
                    next_col += 1
                    result = explore(puzzle, next_row, next_col)
                elif row_index < 2:
                    # go to start of next row
                    next_row += 1
                    next_col = 0
                    result = explore(puzzle, next_row, next_col)
                else:
                    # all done
                    return True
                # did we succesfully fill in the rest of the puzzle?
                if result:
                    # found a solution
                    return True
                else:
                    # undo choice because the recursive call could not fill in the rest of the puzzle
                    puzzle[row_index][col_index] = 0
        return False

def complete_puzzle(puzzle):
    explore(puzzle, 0, 0)

puzzle = [[0, 0, 0], [0, 2, 3], [0, 3, 1]]
print('puzzle: ', puzzle)
complete_puzzle(puzzle)
print('filled: ', puzzle)
print()
puzzle = [[0, 0, 0], [0, 0, 0], [0, 0, 2]]
print('puzzle: ', puzzle)
complete_puzzle(puzzle)
print('filled: ', puzzle)


**Exercise 8** (Longer problem; no solution will be provided) Repeat Exercise 7 for the $9 \times 9$ version
of Sudoku.